
"""
CHECK

- The Missing Value Treatement for the rtgp (runtime in group) variables - espcially for races where NONE of the dogs have a runtime

More Variables :

Total Races from current race
- Races in the last 7/14/28/60/90/180/365/730 days 
- Races won in the 7/14/28/60/90/180/365/730 days
- Races placed in the 7/14/28/60/90/180/365/730 days


"""


# Import the code 


In [1]:

#fileloc  = '/mnt/batch/tasks/shared/LS_root/mounts/clusters/compute-one/code/Users/karan.bhatti'
fileloc  = '/mnt/batch/tasks/shared/LS_root/mounts/clusters/compute-one-8c-32gb/code/Users/karan.bhatti'

import sys
print(sys.path)
print()
#sys.path.append(f'{fileloc}')
print()
#print(sys.path)


['/mnt/batch/tasks/shared/LS_root/mounts/clusters/compute-one-8c-32gb/code/Users/karan.bhatti', '/anaconda/envs/azureml_py38/lib/python38.zip', '/anaconda/envs/azureml_py38/lib/python3.8', '/anaconda/envs/azureml_py38/lib/python3.8/lib-dynload', '', '/anaconda/envs/azureml_py38/lib/python3.8/site-packages', '/anaconda/envs/azureml_py38/lib/python3.8/site-packages/azureml/_project/vendor', '/anaconda/envs/azureml_py38/lib/python3.8/site-packages/azureml/automl/core/_vendor']




In [2]:

class Data(object):
    pass

df = Data
summ = Data


In [3]:
import helpdesk as hd

In [4]:

import pandas as pd, numpy as np
from datetime import datetime


In [5]:
DEBUG = True

In [6]:

pd.set_option('display.max_rows', 50);
pd.set_option('display.max_columns', 50);
pd.set_option('display.width', 200);


In [7]:

#fname_race = 'race_details_2022Q1_2022Q4.csv'
#fname_dogs = 'dog_results_2022Q1_2022Q2.csv'

#race_details = pd.read_csv(f'{fileloc}/{fname_race}')
#dogs_details = pd.read_csv(f'{fileloc}/{fname_dogs}')

#print(race_details.shape)
#print(dogs_details.shape)


## IMPORTING THE RAW DATA AND MERGE

In [8]:

# Importing Dog Race Results

#res_raw = pd.read_csv('C:\\Users\\karan\\Documents\\Data\\racing\\FastTrack\\Past\\dog_results_20211201_20211231.csv')
#df.res_raw = pd.read_csv('C:\\Users\\karan\\Documents\\Data\\racing\\FastTrack\\Past\\dog_results_20220101_20220531.csv')

print(f'{fileloc}/dog_results_2022Q1_2022Q2.csv')

df.res_raw_2021Q1Q4 = pd.read_csv(f'{fileloc}/dog_results_2021Q1_2021Q4.csv')
print(df.res_raw_2021Q1Q4.columns.values.tolist())
print(df.res_raw_2021Q1Q4.shape)

df.res_raw_2022Q1Q2 = pd.read_csv(f'{fileloc}/dog_results_2022Q1_2022Q2.csv')
print(df.res_raw_2022Q1Q2.columns.values.tolist())
print(df.res_raw_2022Q1Q2.shape)



/mnt/batch/tasks/shared/LS_root/mounts/clusters/compute-one-8c-32gb/code/Users/karan.bhatti/dog_results_2022Q1_2022Q2.csv
['@id', 'Place', 'DogName', 'Box', 'Rug', 'Weight', 'StartPrice', 'Handicap', 'Margin1', 'Margin2', 'PIR', 'Checks', 'Comments', 'SplitMargin', 'RunTime', 'Prizemoney', 'RaceId', 'TrainerId', 'TrainerName']
(435715, 19)
['@id', 'Place', 'DogName', 'Box', 'Rug', 'Weight', 'StartPrice', 'Handicap', 'Margin1', 'Margin2', 'PIR', 'Checks', 'Comments', 'SplitMargin', 'RunTime', 'Prizemoney', 'RaceId', 'TrainerId', 'TrainerName']
(320052, 19)


In [9]:

df.res_raw = df.res_raw_2021Q1Q4.append( df.res_raw_2022Q1Q2 )
print(df.res_raw.columns.values.tolist())
print(df.res_raw.shape)


['@id', 'Place', 'DogName', 'Box', 'Rug', 'Weight', 'StartPrice', 'Handicap', 'Margin1', 'Margin2', 'PIR', 'Checks', 'Comments', 'SplitMargin', 'RunTime', 'Prizemoney', 'RaceId', 'TrainerId', 'TrainerName']
(755767, 19)


In [10]:

df.res_raw.loc[:,"DogName"] = df.res_raw["DogName"].str.replace("'","").str.replace(".","")
if DEBUG: print(df.res_raw.shape) 
# 320 052

#if DEBUG: res_raw[ ~(res_raw.Place.isin( ['D','F','N','R','S','T',''])) ].shape # 66608
if DEBUG: print(df.res_raw[ ~(df.res_raw.Place.isin( ['R','S'])) ].shape) 
# 275 864

df.res_raw2 = df.res_raw[ ~(df.res_raw.Place.isin( ['R','S']))  ]
df.res_ = df.res_raw2[~df.res_raw2.Place.isna()]
if DEBUG: print(df.res_.shape) 
# 273 735

del df.res_raw, df.res_raw2

#res_ = res_raw


(755767, 19)
(649399, 19)
(644761, 19)


In [11]:

# Importing Race Details 

#race_raw = pd.read_csv('C:\\Users\\karan\\Documents\\Data\\racing\\FastTrack\\Past\\race_details_20211201_20211231.csv', parse_dates = True)
#df.race_raw = pd.read_csv('C:\\Users\\karan\\Documents\\Data\\racing\\FastTrack\\Past\\race_details_20220101_20220531.csv', parse_dates = True)

print(f'{fileloc}/race_details_2022Q1_2022Q2.csv')

df.race_raw_2022Q1Q2 = pd.read_csv(f'{fileloc}/race_details_2022Q1_2022Q2.csv', parse_dates = True)
print(df.race_raw_2022Q1Q2.columns.values.tolist())
print(df.race_raw_2022Q1Q2.shape)

df.race_raw_2021Q1Q4 = pd.read_csv(f'{fileloc}/race_details_2021Q1_2021Q4.csv', parse_dates = True)
print(df.race_raw_2021Q1Q4.columns.values.tolist())
print(df.race_raw_2021Q1Q4.shape)


/mnt/batch/tasks/shared/LS_root/mounts/clusters/compute-one-8c-32gb/code/Users/karan.bhatti/race_details_2022Q1_2022Q2.csv
['@id', 'RaceNum', 'RaceName', 'RaceTime', 'Distance', 'RaceGrade', 'Track', 'date']
(38562, 8)
['@id', 'RaceNum', 'RaceName', 'RaceTime', 'Distance', 'RaceGrade', 'Track', 'date']
(51833, 8)


In [12]:


df.race_raw = df.race_raw_2021Q1Q4.append( df.race_raw_2022Q1Q2 )
print(df.race_raw.columns.values.tolist())
print(df.race_raw.shape)




df.race_raw['Event_Dt'] = pd.to_datetime(df.race_raw['date']).dt.date

#dtvalct = race_raw.Event_Dt.value_counts()

df.race_raw.loc[df.race_raw.Track.isin(['Murray Bridge (MBR)','Murray Bridge (MBS)']), "Track"] = 'Murray Bridge'
df.race_raw.loc[df.race_raw.Track.isin(['Richmond (RIS)']), "Track"] = 'Richmond'

df.race_raw.loc[df.race_raw.Track.isin(['Sandown (SAP)']), "Track"] = 'Sandown Park'

df.race_raw.loc[df.race_raw.Track.isin(['Meadows (MEP)']), "Track"] = 'Meadows'
df.race_raw.loc[df.race_raw.Track.isin(['The Meadows']), "Track"] = 'Meadows'


if DEBUG : print(df.race_raw.shape)

# 2021
# 52,103 Races

# 2022 upto May
# 21,002 Races

# 2022 Q1/Q2
# 38,562 Races

# 2022 Q1/Q2 + 2021 Q1/Q2/Q3/Q4
# 81,412 Races

df.race_ = df.race_raw[ ~df.race_raw.Track.str.contains('NZ')]

if DEBUG : print(df.race_.shape)

del df.race_raw

#if DEBUG: print(df.race_.dtypes)


['@id', 'RaceNum', 'RaceName', 'RaceTime', 'Distance', 'RaceGrade', 'Track', 'date']
(90395, 8)
(90395, 9)
(81412, 9)


In [84]:

if DEBUG:
    print(df.res_.columns.values.tolist())
    print(df.race_.columns.values.tolist())
    
    print(df.res_.shape)
    print(df.race_.shape)

df.ft_raw = pd.merge(df.res_, df.race_, left_on = 'RaceId', right_on = '@id', how = 'inner', suffixes=('_DOG', '_RACE'))
if DEBUG: print(df.ft_raw.shape)

# 2021
# 333,285

# 2022 upto May
# 132,558

# 2022 Q1/Q2
# 245,827

# 2022 Q1/Q2 + 2021 Q1/Q2/Q3/Q4
# 245,827

#qc_DogName = df.ft_merg.DogName.value_counts()


['@id', 'Place', 'DogName', 'Box', 'Rug', 'Weight', 'StartPrice', 'Handicap', 'Margin1', 'Margin2', 'PIR', 'Checks', 'Comments', 'SplitMargin', 'RunTime', 'Prizemoney', 'RaceId', 'TrainerId', 'TrainerName']
['@id', 'RaceNum', 'RaceName', 'RaceTime', 'Distance', 'RaceGrade', 'Track', 'date', 'Event_Dt']
(644761, 19)
(81412, 9)
(577317, 28)


In [47]:

#summ.ft = hd.describe_df(df.ft)


In [85]:

print(df.ft_raw.columns.values.tolist())


['@id_DOG', 'Place', 'DogName', 'Box', 'Rug', 'Weight', 'StartPrice', 'Handicap', 'Margin1', 'Margin2', 'PIR', 'Checks', 'Comments', 'SplitMargin', 'RunTime', 'Prizemoney', 'RaceId', 'TrainerId', 'TrainerName', '@id_RACE', 'RaceNum', 'RaceName', 'RaceTime', 'Distance', 'RaceGrade', 'Track', 'date', 'Event_Dt']


### Exclude Races with "unplaced" Dogs 

In [87]:

#Clean the fast track prices
df.ft_raw['ft_sp'] = df.ft_raw.StartPrice.apply(lambda x : float(str(x).replace("$","").replace("F","")) )

print( pd.DataFrame(df.ft_raw['Place'].value_counts()).transpose() )
place_to_remove = ['F','T','N','P','B','D']


# Delete races where ft_sp is blank
all_races = df.ft_raw['@id_RACE'].unique().tolist()
print(len(all_races))

# Get Race Lists
blank_ft_sp_races = df.ft_raw[df.ft_raw['ft_sp'].isna()]['@id_RACE'].unique().tolist()
print(len(blank_ft_sp_races))

races_excl_pos = df.ft_raw[df.ft_raw['Place'].isin(place_to_remove)]['@id_RACE'].unique().tolist()
print(len(races_excl_pos))


# Remove races
df.ft = df.ft_raw[ ( ~df.ft_raw['@id_RACE'].isin(blank_ft_sp_races) ) & ( ~df.ft_raw['@id_RACE'].isin(races_excl_pos) ) ]

print(datetime.now())


           1      2      3      4      5      6      7      8     F     T    N   3=   4=   2=   5=   1=   6=    P   7=   B   D
Place  80621  80259  80031  79721  78416  72837  58238  38959  3402  1199  998  476  423  416  373  306  271  222  114  21  14
80909
1391
4407
2022-11-24 07:45:19.613243


In [91]:

print(df.ft.shape)

all_races_f = df.ft['@id_RACE'].unique().tolist()
print(f'RACES : {len(all_races_f)}, RACES RAW : {len(all_races)}')


(541552, 29)
RACES : 75740, RACES RAW : 80909

['@id_DOG', 'Place', 'DogName', 'Box', 'Rug', 'Weight', 'StartPrice', 'Handicap', 'Margin1', 'Margin2', 'PIR', 'Checks', 'Comments', 'SplitMargin', 'RunTime', 'Prizemoney', 'RaceId', 'TrainerId', 'TrainerName', '@id_RACE', 'RaceNum', 'RaceName', 'RaceTime', 'Distance', 'RaceGrade', 'Track', 'date', 'Event_Dt', 'ft_sp']


### Adding variables

In [92]:

# Sort for Easy QC
df.ft.sort_values(by = ['@id_DOG', 'DogName', 'Event_Dt'], inplace = True)


/tmp/ipykernel_11440/837224923.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.ft.sort_values(by = ['@id_DOG', 'DogName', 'Event_Dt'], inplace = True)


In [93]:

#del dognameslist

# DogNames list
dognameslist = df.ft.DogName.unique().tolist()

# Fix the Place Variable
df.ft['position'] = df.ft.Place.apply(lambda x : x.replace("=","") if x in ['1=','2=','3=','4=','5=','6=','7=','8='] else x)

#print(df.ft.position.value_counts())


/tmp/ipykernel_11440/3477695241.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.ft['position'] = df.ft.Place.apply(lambda x : x.replace("=","") if x in ['1=','2=','3=','4=','5=','6=','7=','8='] else x)


In [94]:

print(dognameslist[5])


BIG TOMMY NITRO


In [95]:

df.ft[df.ft['@id_DOG'].isin([108390981])]


,@id_DOG,Place,DogName,Box,Rug,Weight,StartPrice,Handicap,Margin1,Margin2,PIR,Checks,Comments,SplitMargin,RunTime,Prizemoney,RaceId,TrainerId,TrainerName,@id_RACE,RaceNum,RaceName,RaceTime,Distance,RaceGrade,Track,date,Event_Dt,ft_sp,position
4011,108390981,4,DOLLY DOES IT,5.0,5,30.4,$11.00,NaN,6.00,0.14,0,0,B3,NaN,26.43,80.0,626271182,92892,M Elphinstone,626271182,10,GREAT NORTHERN,03:14PM,452m,Mixed 4/5,Devonport,05 Jan 21,2021-01-05,11.0,4
10008,108390981,5,DOLLY DOES IT,4.0,4,30.2,$7.00,NaN,6.00,1.14,0,0,NaN,NaN,26.40,0.0,629797242,92892,M Elphinstone,629797242,10,GREAT NORTHERN,03:09PM,452m,Grade 5,Devonport,12 Jan 21,2021-01-12,7.0,5
16043,108390981,1,DOLLY DOES IT,5.0,5,30.2,$7.00,NaN,5.25,NaN,0,0,NaN,8.1,26.20,1890.0,629797513,92892,M Elphinstone,629797513,4,SUREPICK,01:27PM,452m,Grade 5,Devonport,19 Jan 21,2021-01-19,7.0,1
39726,108390981,7,DOLLY DOES IT,1.0,1,30.0,$7.00,NaN,10.50,2.00,0,0,NaN,NaN,26.48,0.0,638810808,92892,M Elphinstone,638810808,8,LADBROKES,02:22PM,452m,Mixed 4/5 Heat,Devonport,16 Feb 21,2021-02-16,7.0,7
51934,108390981,8,DOLLY DOES IT,8.0,8,29.9,$91.00,NaN,18.50,3.57,0,0,SO,NaN,26.91,0.0,643554706,92892,M Elphinstone,643554706,8,NEW RECRUIT 2012,02:22PM,452m,Grade 4,Devonport,02 Mar 21,2021-03-02,91.0,8


In [96]:

# ADDITIONAL VARIABLES
doglevel = ['@id_DOG']

# Distance and Speed in each race
df.ft['x_dist'] = df.ft['Distance'].apply(lambda x : x.replace('m','')).astype(int)
df.ft['speed'] = (df.ft['x_dist'] / df.ft['RunTime'])*1000

# Flag for getting a total race count
df.ft['f_1'] = 1

# Total Races
df.ft['x_races_tot'] = df.ft.groupby(doglevel).cumcount()


/tmp/ipykernel_11440/3645243109.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.ft['x_dist'] = df.ft['Distance'].apply(lambda x : x.replace('m','')).astype(int)
/tmp/ipykernel_11440/3645243109.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.ft['speed'] = (df.ft['x_dist'] / df.ft['RunTime'])*1000
/tmp/ipykernel_11440/3645243109.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

In [97]:

# Average Speed until prev race
df.ft['speed_prev'] = df.ft.groupby(doglevel)['speed'].shift(1)

df.ft['speed_avg'] = df.ft.groupby(doglevel)['speed_prev'].rolling(window=99, min_periods = 1).mean().reset_index(0,drop=True)
df.ft['speed_min'] = df.ft.groupby(doglevel)['speed_prev'].rolling(window=99, min_periods = 1).min().reset_index(0,drop=True)
df.ft['speed_max'] = df.ft.groupby(doglevel)['speed_prev'].rolling(window=99, min_periods = 1).max().reset_index(0,drop=True)

df.ft['speed_range'] = df.ft['speed_max'] - df.ft['speed_min']

df.ft['speed_std'] = df.ft.groupby(doglevel)['speed_prev'].rolling(window=99, min_periods = 1).std().reset_index(0,drop=True)

print(datetime.now())
      

/tmp/ipykernel_11440/228467171.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.ft['speed_prev'] = df.ft.groupby(doglevel)['speed'].shift(1)
/tmp/ipykernel_11440/228467171.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.ft['speed_avg'] = df.ft.groupby(doglevel)['speed_prev'].rolling(window=99, min_periods = 1).mean().reset_index(0,drop=True)
/tmp/ipykernel_11440/228467171.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer

2022-11-24 07:48:40.793431


/tmp/ipykernel_11440/228467171.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.ft['speed_std'] = df.ft.groupby(doglevel)['speed_prev'].rolling(window=99, min_periods = 1).std().reset_index(0,drop=True)


In [98]:

# ADDITIONAL VARIABLES
grouplevel = ['@id_DOG','x_dist_gp']

# GROUPED X_DIST 
df.ft['x_dist_gp'] = df.ft['x_dist'].apply(lambda x : 300 if x < 330 else 350 if x < 380 else 400 if x < 420 else 450 if x < 485 else 500 if x < 550 else 600 if x < 650 else 700 if x < 750 else -999)
df.ft['x_runtm_gp'] = ( df.ft['RunTime'] / df.ft['x_dist'] ) * df.ft['x_dist_gp']

df.ft['x_rtgp_prev'] = df.ft.groupby(grouplevel)['x_runtm_gp'].shift(1)

#df.ft['x_rtgp_avg'] = df.ft.groupby(grouplevel)['x_rtgp_prev'].rolling(window=99, min_periods = 1).mean().reset_index(drop=True)
df.ft['x_rtgp_avg'] = df.ft.groupby(grouplevel)['x_rtgp_prev'].rolling(window=99, min_periods = 1).mean().values

#df.ft['x_rtgp_min'] = df.ft.groupby(grouplevel)['x_rtgp_prev'].rolling(window=99, min_periods = 1).min().reset_index(drop=True)
df.ft['x_rtgp_min'] = df.ft.groupby(grouplevel)['x_rtgp_prev'].rolling(window=99, min_periods = 1).min().values

#df.ft['x_rtgp_max'] = df.ft.groupby(grouplevel)['x_rtgp_prev'].rolling(window=99, min_periods = 1).max().reset_index(0,drop=True)
df.ft['x_rtgp_max'] = df.ft.groupby(grouplevel)['x_rtgp_prev'].rolling(window=99, min_periods = 1).max().values

df.ft['x_rtgp_range'] = df.ft['x_rtgp_max'] - df.ft['x_rtgp_min']

#df.ft['x_rtgp_std'] = df.ft.groupby(grouplevel)['x_rtgp_prev'].rolling(window=99, min_periods = 1).std().reset_index(0,drop=True)
df.ft['x_rtgp_std'] = df.ft.groupby(grouplevel)['x_rtgp_prev'].rolling(window=99, min_periods = 1).std().values

print(datetime.now())


/tmp/ipykernel_11440/691471815.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.ft['x_dist_gp'] = df.ft['x_dist'].apply(lambda x : 300 if x < 330 else 350 if x < 380 else 400 if x < 420 else 450 if x < 485 else 500 if x < 550 else 600 if x < 650 else 700 if x < 750 else -999)
/tmp/ipykernel_11440/691471815.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.ft['x_runtm_gp'] = ( df.ft['RunTime'] / df.ft['x_dist'] ) * df.ft['x_dist_gp']
/tmp/ipykernel_11440/691471815.py:8: SettingWithCopyWarning: 
A 

2022-11-24 07:48:47.431766


/tmp/ipykernel_11440/691471815.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.ft['x_rtgp_std'] = df.ft.groupby(grouplevel)['x_rtgp_prev'].rolling(window=99, min_periods = 1).std().values


In [99]:

#df.ft.head(25)

#df.ft[ df.ft['@id_DOG'].isin([108390981,108391383,108391431]) ].head(25)

df.ft[ df.ft['x_runtm_gp'].isna()].head()



,@id_DOG,Place,DogName,Box,Rug,Weight,StartPrice,Handicap,Margin1,Margin2,PIR,Checks,Comments,SplitMargin,RunTime,Prizemoney,RaceId,TrainerId,TrainerName,@id_RACE,RaceNum,RaceName,RaceTime,Distance,RaceGrade,Track,date,Event_Dt,ft_sp,position,x_dist,speed,f_1,x_races_tot,speed_prev,speed_avg,speed_min,speed_max,speed_range,speed_std,x_dist_gp,x_runtm_gp,x_rtgp_prev,x_rtgp_avg,x_rtgp_min,x_rtgp_max,x_rtgp_range,x_rtgp_std


In [100]:

print(df.ft.columns.values.tolist())


['@id_DOG', 'Place', 'DogName', 'Box', 'Rug', 'Weight', 'StartPrice', 'Handicap', 'Margin1', 'Margin2', 'PIR', 'Checks', 'Comments', 'SplitMargin', 'RunTime', 'Prizemoney', 'RaceId', 'TrainerId', 'TrainerName', '@id_RACE', 'RaceNum', 'RaceName', 'RaceTime', 'Distance', 'RaceGrade', 'Track', 'date', 'Event_Dt', 'ft_sp', 'position', 'x_dist', 'speed', 'f_1', 'x_races_tot', 'speed_prev', 'speed_avg', 'speed_min', 'speed_max', 'speed_range', 'speed_std', 'x_dist_gp', 'x_runtm_gp', 'x_rtgp_prev', 'x_rtgp_avg', 'x_rtgp_min', 'x_rtgp_max', 'x_rtgp_range', 'x_rtgp_std']


In [101]:

#temp = df.ft.groupby('x_dist').agg( {'@id_RACE':'nunique'} ).reset_index()
#temp.to_csv(f'{fileloc}/race_distance.csv', index = False)
#temp.sort_values(by = 'index').head()


In [102]:

print( len(dognameslist) )

print( len(df.ft['@id_DOG'].unique().tolist()) )

print(df.ft['speed'].isna().sum())
print(df.ft['speed_prev'].isna().sum())


25327
25312
0
25312


In [103]:

print(pd.DataFrame(df.ft.dtypes).transpose())


  @id_DOG   Place DogName      Box    Rug   Weight StartPrice Handicap  Margin1  Margin2     PIR Checks Comments SplitMargin  RunTime Prizemoney RaceId TrainerId TrainerName @id_RACE RaceNum  \
0   int64  object  object  float64  int64  float64     object  float64  float64  float64  object  int64   object     float64  float64    float64  int64     int64      object    int64   int64   

  RaceName RaceTime Distance RaceGrade   Track    date Event_Dt    ft_sp position x_dist    speed    f_1 x_races_tot speed_prev speed_avg speed_min speed_max speed_range speed_std x_dist_gp  \
0   object   object   object    object  object  object   object  float64   object  int64  float64  int64       int64    float64   float64   float64   float64     float64   float64     int64   

  x_runtm_gp x_rtgp_prev x_rtgp_avg x_rtgp_min x_rtgp_max x_rtgp_range x_rtgp_std  
0    float64     float64    float64    float64    float64      float64    float64  


In [104]:

print(pd.DataFrame(df.ft.isna().sum()).transpose())


   @id_DOG  Place  DogName  Box  Rug  Weight  StartPrice  Handicap  Margin1  Margin2    PIR  Checks  Comments  SplitMargin  RunTime  Prizemoney  RaceId  TrainerId  TrainerName  @id_RACE  RaceNum  \
0        0      0        0    0    0       1           0    541552       19    75886  72204       0    261971        91093        0      168479       0          0            0         0        0   

   RaceName  RaceTime  Distance  RaceGrade  Track  date  Event_Dt  ft_sp  position  x_dist  speed  f_1  x_races_tot  speed_prev  speed_avg  speed_min  speed_max  speed_range  speed_std  x_dist_gp  \
0        15         0         0          0      0     0         0      0         0       0      0    0            0       25312      25312      25312      25312        25312      49246          0   

   x_runtm_gp  x_rtgp_prev  x_rtgp_avg  x_rtgp_min  x_rtgp_max  x_rtgp_range  x_rtgp_std  
0           0        72242       72242       72242       72242         72242      129743  



### Treat Missing Values


In [105]:

# All Columns Realted to Average Speed

df.ft['x_no_prev_races'] = df.ft['speed_prev'].apply( lambda x : 1 if pd.isna(x) else 0)

print( df.ft['x_no_prev_races'].value_counts() )

df.ft['x_speed_prev'] = df.ft['speed_prev'].fillna(0)
df.ft['x_speed_avg'] = df.ft['speed_avg'].fillna(0)
df.ft['x_speed_min'] = df.ft['speed_min'].fillna(0)
df.ft['x_speed_max'] = df.ft['speed_max'].fillna(0)

df.ft['x_speed_range'] = df.ft['speed_range'].fillna(-1)

df.ft['x_speed_std'] = df.ft['speed_std'].fillna(0)
df.ft['x_speed_std'] = df.ft.apply(lambda x : -1 if x.x_no_prev_races == 1 else x.x_speed_std, axis = 1  )

print(datetime.now())


/tmp/ipykernel_11440/2886457866.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.ft['x_no_prev_races'] = df.ft['speed_prev'].apply( lambda x : 1 if pd.isna(x) else 0)
/tmp/ipykernel_11440/2886457866.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.ft['x_speed_prev'] = df.ft['speed_prev'].fillna(0)
/tmp/ipykernel_11440/2886457866.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

0    516240
1     25312
Name: x_no_prev_races, dtype: int64
2022-11-24 07:48:55.092779


/tmp/ipykernel_11440/2886457866.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.ft['x_speed_std'] = df.ft.apply(lambda x : -1 if x.x_no_prev_races == 1 else x.x_speed_std, axis = 1  )


In [113]:

# All Columns Related to RunTime in Group

df.ft['x_no_prev_r_in_dist'] = df.ft['x_rtgp_prev'].apply( lambda x : 1 if pd.isna(x) else 0)

print( df.ft['x_no_prev_r_in_dist'].value_counts() )

print(datetime.now())


0    469310
1     72242
Name: x_no_prev_r_in_dist, dtype: int64
2022-11-24 07:49:10.361176


/anaconda/envs/azureml_py38/lib/python3.8/site-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [119]:

rtgp_mvt = df.ft.groupby('@id_RACE').agg({ 'x_rtgp_prev':'mean', 'x_rtgp_avg':'mean', 'x_rtgp_min':'mean', 'x_rtgp_max':'mean', 'x_rtgp_range':'mean', 'x_rtgp_std':'mean'})\
            .reset_index()\
            .rename(columns = {'x_rtgp_prev':'mvt_rtgp_prev', 'x_rtgp_avg':'mvt_rtgp_avg', 'x_rtgp_min':'mvt_rtgp_min', 'x_rtgp_max':'mvt_rtgp_max', 'x_rtgp_range':'mvt_rtgp_range', 'x_rtgp_std':'mvt_rtgp_std'})

print(pd.DataFrame(rtgp_mvt.isna().sum()).transpose())


In [130]:

print(pd.DataFrame(df.ft.isna().sum()).transpose())
print()
print()
print()

print(df.ft.shape)
df.ft_mvt = df.ft.merge(rtgp_mvt, on = '@id_RACE', how = 'left')
print(df.ft_mvt.shape)

df.ft_mvt['x_rtgp_prev'].fillna(df.ft_mvt['mvt_rtgp_prev'], inplace=True)

df.ft_mvt['x_rtgp_avg'].fillna(df.ft_mvt['mvt_rtgp_avg'], inplace=True)
df.ft_mvt['x_rtgp_min'].fillna(df.ft_mvt['mvt_rtgp_min'], inplace=True)
df.ft_mvt['x_rtgp_max'].fillna(df.ft_mvt['mvt_rtgp_max'], inplace=True)

df.ft_mvt['x_rtgp_range'].fillna(df.ft_mvt['mvt_rtgp_range'], inplace=True)
df.ft_mvt['x_rtgp_std'].fillna(df.ft_mvt['mvt_rtgp_std'], inplace=True)

print()
print()
print()
print(pd.DataFrame(df.ft_mvt.isna().sum()).transpose())


# MIGHT NEED TO CHECK THIS AGAIN
df.ft_mvt['x_rtgp_prev'].fillna(0, inplace=True)

df.ft_mvt['x_rtgp_avg'].fillna(0, inplace=True)
df.ft_mvt['x_rtgp_min'].fillna(0, inplace=True)
df.ft_mvt['x_rtgp_max'].fillna(0, inplace=True)

df.ft_mvt['x_rtgp_range'].fillna(0, inplace=True)
df.ft_mvt['x_rtgp_std'].fillna(0, inplace=True)

print()
print()
print()
print(pd.DataFrame(df.ft_mvt.isna().sum()).transpose())


   @id_DOG  Place  DogName  Box  Rug  Weight  StartPrice  Handicap  Margin1  Margin2    PIR  Checks  Comments  SplitMargin  RunTime  Prizemoney  RaceId  TrainerId  TrainerName  @id_RACE  RaceNum  \
0        0      0        0    0    0       1           0    541552       19    75886  72204       0    261971        91093        0      168479       0          0            0         0        0   

   RaceName  RaceTime  Distance  RaceGrade  ...  speed  f_1  x_races_tot  speed_prev  speed_avg  speed_min  speed_max  speed_range  speed_std  x_dist_gp  x_runtm_gp  x_rtgp_prev  x_rtgp_avg  \
0        15         0         0          0  ...      0    0            0       25312      25312      25312      25312        25312      49246          0           0        72242       72242   

   x_rtgp_min  x_rtgp_max  x_rtgp_range  x_rtgp_std  x_no_prev_races  x_speed_prev  x_speed_avg  x_speed_min  x_speed_max  x_speed_range  x_speed_std  x_no_prev_r_in_dist  
0       72242       72242         72242    

In [131]:

#df.ft_mvt[ df.ft_mvt.mvt_rtgp_prev.isna() | df.ft_mvt.x_rtgp_prev.isna()  ].head(25)


df.ft_mvt[ df.ft_mvt['@id_RACE'].isin([626271182, 626218628, 626240593])].sort_values(by = ['@id_RACE','Place']).head(25)


,@id_DOG,Place,DogName,Box,Rug,Weight,StartPrice,Handicap,Margin1,Margin2,PIR,Checks,Comments,SplitMargin,RunTime,Prizemoney,RaceId,TrainerId,TrainerName,@id_RACE,RaceNum,RaceName,RaceTime,Distance,RaceGrade,...,speed_max,speed_range,speed_std,x_dist_gp,x_runtm_gp,x_rtgp_prev,x_rtgp_avg,x_rtgp_min,x_rtgp_max,x_rtgp_range,x_rtgp_std,x_no_prev_races,x_speed_prev,x_speed_avg,x_speed_min,x_speed_max,x_speed_range,x_speed_std,x_no_prev_r_in_dist,mvt_rtgp_prev,mvt_rtgp_avg,mvt_rtgp_min,mvt_rtgp_max,mvt_rtgp_range,mvt_rtgp_std
16770,251076212,1,BURN OUT BOY,5.0,5,33.2,$2.60F,NaN,1.00,NaN,211,0,M21c1 Sw,6.62,22.23,1100.0,626218628,68894,G Price,626218628,3,Try The Chef's Special,06:52PM,380m,Grade 6,...,NaN,NaN,NaN,400,23.400000,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,-1.0,-1.0,1,NaN,NaN,NaN,NaN,NaN,NaN
176710,415996356,2,SCARY FUN,3.0,3,27.2,$4.00,NaN,1.00,1.14,332,0,Mc3c3c2,6.77,22.31,330.0,626218628,112106,L Isaac,626218628,3,Try The Chef's Special,06:52PM,380m,Grade 6,...,NaN,NaN,NaN,400,23.484211,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,-1.0,-1.0,1,NaN,NaN,NaN,NaN,NaN,NaN
31598,295446742,3,METRE EATER,2.0,2,33.1,$9.90,NaN,4.25,3.00,744,0,Sc7c44,6.88,22.52,165.0,626218628,72510,B McPherson,626218628,3,Try The Chef's Special,06:52PM,380m,Grade 6,...,NaN,NaN,NaN,400,23.705263,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,-1.0,-1.0,1,NaN,NaN,NaN,NaN,NaN,NaN
7210,219044234,4,REPEAT SET,8.0,8,30.4,$12.00,NaN,5.50,1.43,656,0,M65w6,6.84,22.62,55.0,626218628,127397,C Bainbridge,626218628,3,Try The Chef's Special,06:52PM,380m,Grade 6,...,NaN,NaN,NaN,400,23.810526,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,-1.0,-1.0,1,NaN,NaN,NaN,NaN,NaN,NaN
14750,246692403,5,KABARANGI,1.0,1,24.1,$40.60,NaN,5.75,0.29,565,0,Q5c65,6.84,22.64,0.0,626218628,291086,T Mitchem,626218628,3,Try The Chef's Special,06:52PM,380m,Grade 6,...,NaN,NaN,NaN,400,23.831579,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,-1.0,-1.0,1,NaN,NaN,NaN,NaN,NaN,NaN
5,108391383,6,SIMPLY GIFTED,6.0,6,24.8,$46.50,NaN,7.25,1.43,877,0,Sc8c77,6.93,22.74,0.0,626218628,291086,T Mitchem,626218628,3,Try The Chef's Special,06:52PM,380m,Grade 6,...,NaN,NaN,NaN,400,23.936842,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,-1.0,-1.0,1,NaN,NaN,NaN,NaN,NaN,NaN
10897,231587263,7,ABBEY SIOBHAN,4.0,4,30.8,$5.60,NaN,9.00,1.71,123,0,Q12cc3,6.59,22.86,0.0,626218628,102996,M Stewart,626218628,3,Try The Chef's Special,06:52PM,380m,Grade 6,...,NaN,NaN,NaN,400,24.063158,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,-1.0,-1.0,1,NaN,NaN,NaN,NaN,NaN,NaN
28145,289026900,8,CROWN BENJI,7.0,9,32.4,$6.80,NaN,18.25,9.29,488,0,M48to8 I-21,6.82,23.51,0.0,626218628,83951,R Noden,626218628,3,Try The Chef's Special,06:52PM,380m,Grade 6,...,NaN,NaN,NaN,400,24.747368,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,-1.0,-1.0,1,NaN,NaN,NaN,NaN,NaN,NaN
216289,434800047,1,IVY CONNIVING,8.0,8,28.2,$2.20F,NaN,8.75,NaN,111,0,111,8.20,22.55,1600.0,626240593,108360,S Sultana,626240593,6,STEELINE RIVERSTONE,04:52PM,400m,Mixed 4/5,...,NaN,NaN,NaN,400,22.550000,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,-1.0,-1.0,1,NaN,NaN,NaN,NaN,NaN,NaN
17133,253088672,2,SHIMMY SPARKLES,3.0,3,27.0,$19.00,NaN,8.75,8.71,544,0,544,8.50,23.16,500.0,626240593,114260,L Azzopardi,626240593,6,STEELINE RIVERSTONE,04:52PM,400m,Mixed 4/5,...,NaN,NaN,NaN,400,23.160000,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,-1.0,-1.0,1,NaN,NaN,NaN,NaN,NaN,NaN


In [135]:

print(df.ft_mvt.describe())


            @id_DOG            Box            Rug         Weight  Handicap        Margin1        Margin2         Checks    SplitMargin        RunTime      Prizemoney        RaceId      TrainerId  \
count  5.415520e+05  541552.000000  541552.000000  541551.000000       0.0  541533.000000  465666.000000  541552.000000  450459.000000  541552.000000   373073.000000  5.415520e+05  541552.000000   
mean   4.525647e+08       4.475832       4.581804      29.920749       NaN       6.437371       2.068106       0.573454       7.303363      24.427979      428.568709  7.292605e+08  109679.400242   
std    1.121930e+08       2.346395       2.433653       3.323094       NaN       8.956389       8.143211       1.643539       2.527004       4.983891     3005.503768  5.831714e+07   76370.760651   
min    1.083910e+08       1.000000       1.000000      18.900000       NaN       0.000000       0.000000       0.000000       0.000000      10.000000        0.000000  6.238619e+08    6418.000000   
25%    3.8

In [136]:

print( df.ft_mvt.columns.values.tolist() )


['@id_DOG', 'Place', 'DogName', 'Box', 'Rug', 'Weight', 'StartPrice', 'Handicap', 'Margin1', 'Margin2', 'PIR', 'Checks', 'Comments', 'SplitMargin', 'RunTime', 'Prizemoney', 'RaceId', 'TrainerId', 'TrainerName', '@id_RACE', 'RaceNum', 'RaceName', 'RaceTime', 'Distance', 'RaceGrade', 'Track', 'date', 'Event_Dt', 'ft_sp', 'position', 'x_dist', 'speed', 'f_1', 'x_races_tot', 'speed_prev', 'speed_avg', 'speed_min', 'speed_max', 'speed_range', 'speed_std', 'x_dist_gp', 'x_runtm_gp', 'x_rtgp_prev', 'x_rtgp_avg', 'x_rtgp_min', 'x_rtgp_max', 'x_rtgp_range', 'x_rtgp_std', 'x_no_prev_races', 'x_speed_prev', 'x_speed_avg', 'x_speed_min', 'x_speed_max', 'x_speed_range', 'x_speed_std', 'x_no_prev_r_in_dist', 'mvt_rtgp_prev', 'mvt_rtgp_avg', 'mvt_rtgp_min', 'mvt_rtgp_max', 'mvt_rtgp_range', 'mvt_rtgp_std']


In [137]:

print(datetime.now().date())


2022-11-24


In [139]:

# 'TrainerName', 'Track' have been dummified
cols_for_drop = ['Box','Rug','Weight','Handicap','Margin1','Margin2','PIR','Checks','Comments','SplitMargin','Prizemoney','RaceId','TrainerId','RaceTime','Distance','RaceGrade','date', 'f_1','speed',\
                'Place','StartPrice'\
                ,'speed_prev', 'speed_avg', 'speed_min', 'speed_max', 'speed_range', 'speed_std'\
                , 'DogName', 'TrainerName', 'RaceNum', 'RaceName', 'Track', 'Event_Dt', 'position','RunTime'\
                , 'x_dist','ft_sp', 'x_dist'\
                , 'mvt_rtgp_prev', 'mvt_rtgp_avg', 'mvt_rtgp_min' ,'mvt_rtgp_max' ,'mvt_rtgp_range', 'mvt_rtgp_std']

## SHOULD BE THE LAST STEP OR a Separate Code
#df.ft1 = df.ft.copy()
exp_ds = df.ft_mvt.drop(columns = cols_for_drop)

# removed the Categorical columns from the original datast, adds the dummy variables, creates a new dataset
print(exp_ds.columns)


Index(['@id_DOG', '@id_RACE', 'x_races_tot', 'x_dist_gp', 'x_runtm_gp', 'x_rtgp_prev', 'x_rtgp_avg', 'x_rtgp_min', 'x_rtgp_max', 'x_rtgp_range', 'x_rtgp_std', 'x_no_prev_races', 'x_speed_prev',
       'x_speed_avg', 'x_speed_min', 'x_speed_max', 'x_speed_range', 'x_speed_std', 'x_no_prev_r_in_dist'],
      dtype='object')


In [140]:

exp_ds.head()


,@id_DOG,@id_RACE,x_races_tot,x_dist_gp,x_runtm_gp,x_rtgp_prev,x_rtgp_avg,x_rtgp_min,x_rtgp_max,x_rtgp_range,x_rtgp_std,x_no_prev_races,x_speed_prev,x_speed_avg,x_speed_min,x_speed_max,x_speed_range,x_speed_std,x_no_prev_r_in_dist
0,108390981,626271182,0,450,26.313053,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1,0.000000,0.000000,0.000000,0.000000,-1.000000,-1.000000,1
1,108390981,629797242,1,450,26.283186,26.313053,26.313053,26.313053,26.313053,0.000000,0.000000,0,17101.778282,17101.778282,17101.778282,17101.778282,0.000000,0.000000,0
2,108390981,629797513,2,450,26.084071,26.283186,26.298119,26.283186,26.313053,0.029867,0.021119,0,17121.212121,17111.495202,17101.778282,17121.212121,19.433839,13.741799,0
3,108390981,638810808,3,450,26.362832,26.084071,26.226770,26.084071,26.313053,0.228982,0.124480,0,17251.908397,17158.299600,17101.778282,17251.908397,150.130115,81.647864,0
4,108390981,643554706,4,450,26.790929,26.362832,26.260785,26.084071,26.362832,0.278761,0.122305,0,17069.486405,17136.096301,17069.486405,17251.908397,182.421992,80.101155,0


### EXPORTING THE SPEED PROFILE csv

In [141]:

#keep_cols_for_export = ['@id_DOG', 'DogName', '@id_RACE','RaceName','Track','RaceGrade','Distance','TrainerName','RunTime','Place','x_races_tot']
#'Event_Dt', 

cols_for_drop = []

todaydt = datetime.now().date()
exp_fname = f'dataprep_speed_profile_{todaydt}.csv'



In [142]:

print(exp_ds.columns.values.tolist())


['@id_DOG', '@id_RACE', 'x_races_tot', 'x_dist_gp', 'x_runtm_gp', 'x_rtgp_prev', 'x_rtgp_avg', 'x_rtgp_min', 'x_rtgp_max', 'x_rtgp_range', 'x_rtgp_std', 'x_no_prev_races', 'x_speed_prev', 'x_speed_avg', 'x_speed_min', 'x_speed_max', 'x_speed_range', 'x_speed_std', 'x_no_prev_r_in_dist']


In [143]:

exp_ds.head()


,@id_DOG,@id_RACE,x_races_tot,x_dist_gp,x_runtm_gp,x_rtgp_prev,x_rtgp_avg,x_rtgp_min,x_rtgp_max,x_rtgp_range,x_rtgp_std,x_no_prev_races,x_speed_prev,x_speed_avg,x_speed_min,x_speed_max,x_speed_range,x_speed_std,x_no_prev_r_in_dist
0,108390981,626271182,0,450,26.313053,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1,0.000000,0.000000,0.000000,0.000000,-1.000000,-1.000000,1
1,108390981,629797242,1,450,26.283186,26.313053,26.313053,26.313053,26.313053,0.000000,0.000000,0,17101.778282,17101.778282,17101.778282,17101.778282,0.000000,0.000000,0
2,108390981,629797513,2,450,26.084071,26.283186,26.298119,26.283186,26.313053,0.029867,0.021119,0,17121.212121,17111.495202,17101.778282,17121.212121,19.433839,13.741799,0
3,108390981,638810808,3,450,26.362832,26.084071,26.226770,26.084071,26.313053,0.228982,0.124480,0,17251.908397,17158.299600,17101.778282,17251.908397,150.130115,81.647864,0
4,108390981,643554706,4,450,26.790929,26.362832,26.260785,26.084071,26.362832,0.278761,0.122305,0,17069.486405,17136.096301,17069.486405,17251.908397,182.421992,80.101155,0


In [144]:

exp_ds.describe()


,@id_DOG,@id_RACE,x_races_tot,x_dist_gp,x_runtm_gp,x_rtgp_prev,x_rtgp_avg,x_rtgp_min,x_rtgp_max,x_rtgp_range,x_rtgp_std,x_no_prev_races,x_speed_prev,x_speed_avg,x_speed_min,x_speed_max,x_speed_range,x_speed_std,x_no_prev_r_in_dist
count,5.415520e+05,5.415520e+05,541552.000000,541552.000000,541552.000000,541552.000000,541552.000000,541552.000000,541552.000000,541552.000000,541552.000000,541552.000000,541552.000000,541552.000000,541552.000000,541552.000000,541552.000000,541552.000000,541552.000000
mean,4.525647e+08,7.292605e+08,17.606673,416.071495,24.355320,24.081394,24.052082,23.716280,24.498180,0.781900,0.307835,0.046740,16304.781328,16302.481999,15819.876758,16700.819115,880.895617,255.382105,0.133398
std,1.121930e+08,5.831714e+07,15.270459,78.482718,4.673255,5.255971,4.932204,4.891811,5.006750,0.696638,0.200511,0.211081,3634.443576,3624.288110,3536.112893,3740.556099,694.980845,201.751293,0.340005
min,1.083910e+08,6.238619e+08,0.000000,300.000000,9.950249,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,-1.000000,0.000000
25%,3.836042e+08,6.802578e+08,6.000000,350.000000,20.745455,20.628788,20.669554,20.406061,21.101122,0.355556,0.212360,0.000000,16802.610114,16880.689318,16326.530612,17227.609812,542.358858,183.626765,0.000000
50%,4.517679e+08,7.293859e+08,14.000000,400.000000,23.703704,23.661538,23.596078,23.251948,24.115789,0.730752,0.286859,0.000000,17088.399606,17104.816404,16617.981175,17509.727626,856.317173,246.941234,0.000000
75%,5.271657e+08,7.792436e+08,26.000000,450.000000,27.240139,27.093968,26.770302,26.447727,27.302784,1.090000,0.372412,0.000000,17351.069983,17291.853075,16856.392294,17743.702081,1147.858644,315.568817,0.000000
max,2.145360e+09,8.292517e+08,120.000000,700.000000,384.582524,61.747368,46.248951,46.248951,61.747368,39.167368,10.132302,1.000000,40200.000000,40200.000000,40200.000000,40200.000000,23826.557771,16614.431948,1.000000


In [145]:

exp_ds.to_csv(f'{fileloc}/{exp_fname}', index = False)


In [146]:

print('THE END')


THE END
